## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
import pandas as pd
import numpy as np
import requests
import gmaps

from config import g_key
gmaps.configure(api_key=g_key)


In [2]:
city_data_df = pd.read_csv("Resources/WeatherPy_database.csv")
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


clean_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & # Create Filtered Dataframe
                                (city_data_df["Min Temp"] >= min_temp)]

if clean_city_data_df.isna().count()[0] != 0: # If null values, drop them 
    clean_city_data_df.dropna()

    
hotel_df = clean_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = np.nan 
hotel_df.head()

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 82


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bubaque,GW,75.51,few clouds,11.2833,-15.8333,NaN
7,Georgetown,MY,80.53,few clouds,5.4112,100.3354,NaN
13,Sao Felix Do Xingu,BR,80.69,overcast clouds,-6.6447,-51.9950,NaN
16,Marsh Harbour,BS,76.87,heavy intensity rain,26.5412,-77.0636,NaN
23,Carutapera,BR,81.03,overcast clouds,-1.1950,-46.0200,NaN


In [3]:
print("Beginning Data Retrieval     ")
print("-----------------------------")

parameters = {"radius": 5000, "type": "lodging", "key": g_key} # json parameters list for get request
nullers = 0

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    hotels = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=parameters).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] # if there is a hotel in the radius
        
    except (IndexError):
        nullers += 1

clean_hotel_df = hotel_df.dropna().drop_duplicates()

clean_hotel_df.to_csv("Resources/WeatherPy_vacation.csv", index_label="Hotel_ID") # export to csv

print(f"{nullers} Cities did not have hotels")
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
32 Cities did not have hotels
-----------------------------
Data Retrieval Complete      
-----------------------------


In [4]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bubaque,GW,75.51,few clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
7,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
13,Sao Felix Do Xingu,BR,80.69,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
16,Marsh Harbour,BS,76.87,heavy intensity rain,26.5412,-77.0636,Abaco Beach Resort
23,Carutapera,BR,81.03,overcast clouds,-1.1950,-46.0200,Lidera
...,...,...,...,...,...,...,...
1729,Ibra,OM,81.09,clear sky,22.6906,58.5334,مزرعة راحة النفوس المنزفة
1739,Tambura,SS,73.81,overcast clouds,5.6000,27.4667,Rikito Lodge
1742,Trairi,BR,79.99,overcast clouds,-3.2778,-39.2689,janelas do mar
1776,Cotonou,BJ,80.58,few clouds,6.3654,2.4183,Novotel Cotonou Orisha


In [5]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))